In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import imutils
import time

In [2]:
CLASS_LABELS  = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', "Surprise"]
CLASS_LABELS_EMOJIS = ["👿", "🤢" , "😱" , "😊" , "😐 ", "😔" , "😲" ]

In [3]:
model = tf.keras.models.load_model("Bin/EmoDetection.h5")
face_cascade = cv2.CascadeClassifier('Bin/haarcascade_frontalface_default.xml')

In [17]:
def getEmotion(img) : 
    img = cv2.resize(img, (48, 48))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = np.stack((img,)*3, axis=-1)
    img = tf.keras.applications.densenet.preprocess_input(img)
    img = img/255.0
    res = model.predict(img.reshape(1, 48, 48, 3))[0]
    id = np.argmax(res)
    return {
        "class_id" : id,
        "class_name" : CLASS_LABELS[id],
        "class_emoji" : CLASS_LABELS_EMOJIS[id],
        "dist" : res
    }

def getEmotionFromUrl(url) : 
  return getEmotion(imutils.url_to_image(url))

getEmotion(cv2.imread("Images/im105.png"))

1/1 [==============================] - 0s 84ms/step


{'class_id': 4,
 'class_name': 'Neutral',
 'class_emoji': '😐 ',
 'dist': array([5.8334600e-02, 1.7989310e-04, 1.5415937e-01, 7.8443093e-03,
        4.9347579e-01, 2.8063637e-01, 5.3696819e-03], dtype=float32)}

In [15]:
vid = cv2.VideoCapture(0)

try :
    count = -1
    while(True):
        count += 1
        ret, frame = vid.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces) > 0 : 
            x,y,w,h = faces[0]
            frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            img = frame[y:y+h, x:x+w]
            if count%10==0 : 
                emo = getEmotion(img)
                cv2.imshow("face_detected", img)
                count = 0
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        time.sleep(0.1)
        
except Exception as e: 
    print(e)
finally : 
    vid.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 80ms/step
{'class_id': 4, 'class_name': 'Neutral', 'class_emoji': '😐 ', 'dist': array([1.8371396e-02, 6.3337907e-06, 6.1851162e-03, 1.3996365e-02,
       9.4430429e-01, 1.4520199e-02, 2.6163855e-03], dtype=float32)}
1/1 [==============================] - 0s 53ms/step
{'class_id': 0, 'class_name': 'Anger', 'class_emoji': '👿', 'dist': array([0.54877645, 0.00109404, 0.17939548, 0.01348808, 0.15963964,
       0.0509445 , 0.04666177], dtype=float32)}
1/1 [==============================] - 0s 69ms/step
{'class_id': 2, 'class_name': 'Fear', 'class_emoji': '😱', 'dist': array([6.7991689e-02, 2.7256733e-04, 5.1231080e-01, 1.0020679e-02,
       2.9589745e-01, 5.9413791e-02, 5.4092962e-02], dtype=float32)}
1/1 [==============================] - 0s 47ms/step
{'class_id': 3, 'class_name': 'Happy', 'class_emoji': '😊', 'dist': array([1.8892117e-03, 1.5805486e-04, 3.9178110e-04, 9.9534816e-01,
       6.6153583e-04, 9.4469503e-04, 6.0652039e-04], dtype=float32)}